# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

# Parse First Text File

In [2]:
file_path = 'mathwell_questions2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100) #past percentage was 54.88
print(result1)
print(len(df1), total_questions)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
Percentage of Questions with Executable Code: 55.395472995209715
15149 27347


In [3]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 6.9311505709947845


In [4]:
df1 = df1.drop_duplicates(subset='question')

In [5]:
df1

,question,solution,answer,topic
0,A baker baked 120 cakes. He sold 40 on the fir...,def solution():\n\n # The baker started wit...,30,Animals and Wildlife
1,Naruto has 300 ramen noodles. He eats 20 ramen...,def solution():\n #Naruto has 300 ramen noo...,200,Naruto
2,A 5th-grade class is doing a science project. ...,def solution():\n #20 students are in the c...,60,DIY Science Projects
3,A mummy is 3000 years old. \n\nHis sarcophagus...,def solution():\n #The mummy is 3000 years ...,4000,Mummies
4,The volleyball team has 12 players. \n4 of the...,def solution():\n #Total number of players\...,0,volleyball
...,...,...,...,...
15143,A phoenix is a mythical bird that is reborn fr...,def solution():\n #The first phoenix was bo...,1005500,Mythology
15144,"Ratatouille, the rat, ate 12 slices of pizza. ...",def solution():\n #Ratatouille ate 12 slice...,0,Ratatouille
15145,A detective is investigating a murder. He inte...,def solution():\n #Total people interviewed...,20,Detectives and Mystery
15146,Shrek ate 30 onions. 10 of the onions were gre...,def solution():\n #Shrek ate 30 onions\n ...,0,Shrek


# Parse Second Text File

In [6]:
file_path = 'mathwell_questions_no_topic.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
print(result1)
print(len(df2), total_questions)

Percentage of Questions with Executable Code: 46.500577886220626
7242 15574


In [7]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 4.901960784313726


In [8]:
df2 = df2.drop_duplicates(subset='question')

In [9]:
df2

,question,solution,answer,topic
0,2000 people in a stadium. 1/4 of the people ar...,def solution():\n #Total people in the stad...,850.0,unspecified
1,The first 500 people in line for the Comic-Con...,def solution():\n #The first 500 people in ...,1000,unspecified
2,There are 120000 spiders in a spider farm. 100...,def solution():\n #There were 120000 spider...,100000,unspecified
3,A baker bakes 100 pies. He sells 20 of them on...,def solution():\n #Baker bakes 100 pies\n ...,10,unspecified
4,A 747 can hold 400 people. \nA 777 can hold 30...,def solution():\n #A 747 can hold 400 peopl...,4000,unspecified
...,...,...,...,...
7232,400 people are at a rally. 200 of them are wea...,def solution():\n #Total people at the rall...,100,unspecified
7233,20000 people live in a small town. 2000 of the...,def solution():\n #12000 people live in a s...,0,unspecified
7237,A baker is baking 1000 cookies. He bakes 200 c...,def solution():\n #The baker is baking 1000...,-400,unspecified
7238,200 people are at a rally. 1/4 of them are wea...,def solution():\n\n #Total number of people...,<class 'int'>,unspecified


# Concat DFs

In [10]:
extra_qs = pd.read_csv('data/extra_qs.csv')
df = pd.concat([df1, df2, extra_qs])
df = df.drop_duplicates(subset='question')
df = df.sample(frac = 1, random_state = 42)
df = df[['question', 'solution', 'answer', 'topic']]
#df = df[:17692]
#df.to_csv("data/sgsm_unannotated.csv")

In [11]:
df ## last run had 17692

,question,solution,answer,topic
10739,The Egyptians built 100 pyramids. 20 of them a...,def solution():\n #Total number of pyramids...,15,Ancient Civilizations
5852,A 2-liter bottle of soda has 80 calories. A 1....,def solution():\n #A 2-liter bottle of soda...,400,Health and Nutrition
1910,There are 20000 apples in an orchard. 2000 of ...,def solution():\n #There are 20000 apples i...,15000,unspecified
7809,There are 1000 pages in a Mystery Stories book...,def solution():\n #Total pages in the book\...,500,Mystery Stories
14961,1000 Minions are in a room. 200 of them are we...,def solution():\n #1000 Minions are in a ro...,1000,Minions
...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)"


In [12]:
#df = pd.read_csv('data/sgsm_unannotated.csv')
def parse_solution(df):
    if "\nBe" in df['solution']:
        solution = df['solution'].split("\nBe")[0]
    if "##" in df['solution']:
        solution = df['solution'].split("##")[0]
    else:
        solution = df['solution']
    return solution
df['solution'] = df.apply(parse_solution, axis =1)
df.to_csv("data/sgsm_unannotated.csv")
sgsm_train = pd.read_csv('data/sgsm_train.csv')
# sgsm_train = sgsm_train.drop_duplicates(subset= 'question')
# sgsm_train.to_csv('data/sgsm_train.csv')
sgsm_train['subset'] = 'sgsm_train'
df['subset'] = 'sgsm_unannotated'
sgsm = pd.concat([sgsm_train, df])
sgsm = sgsm[['question', 'solution', 'answer', 'topic', 'subset']]
#sgsm.to_csv('data/sgsm.csv')
#sgsm = sgsm.drop_duplicates(subset = ['question', 'subset'])
sgsm

,question,solution,answer,topic,subset
0,"LeBron James is 6'9"" tall. 1 inch is 2.54 cm. ...","def solution():\n #LeBron James is 6'9"" tal...",205.74,basketball,sgsm_train
1,Captain America has 1200 ryo. He spends 400 ry...,def solution():\n #Captain America started ...,300.0,Captain America,sgsm_train
2,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream,sgsm_train
3,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball,sgsm_train
4,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman,sgsm_train
...,...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts,sgsm_unannotated
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures,sgsm_unannotated
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens,sgsm_unannotated
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)",sgsm_unannotated


In [13]:
sgsm = sgsm.drop_duplicates(subset = 'question')
sgsm[sgsm['subset']=='sgsm_unannotated'].to_csv('data/sgsm_unannotated.csv')
sgsm

,question,solution,answer,topic,subset
0,"LeBron James is 6'9"" tall. 1 inch is 2.54 cm. ...","def solution():\n #LeBron James is 6'9"" tal...",205.74,basketball,sgsm_train
1,Captain America has 1200 ryo. He spends 400 ry...,def solution():\n #Captain America started ...,300.0,Captain America,sgsm_train
2,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream,sgsm_train
3,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball,sgsm_train
4,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman,sgsm_train
...,...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts,sgsm_unannotated
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures,sgsm_unannotated
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens,sgsm_unannotated
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)",sgsm_unannotated


In [14]:
sgsm.to_csv('data/sgsm.csv', index=False)
sgsm.to_csv('sgsm.csv', index=False)
sgsm['subset'].value_counts()

subset
sgsm_unannotated    18397
sgsm_train           2093
Name: count, dtype: int64

In [15]:
sgsm_train = pd.read_csv('data/sgsm_train.csv')
sgsm_train.drop_duplicates(subset= ['question', 'solution'])

,Unnamed: 0,question,solution,answer,topic
0,0,"LeBron James is 6'9"" tall. 1 inch is 2.54 cm. ...","def solution():\n #LeBron James is 6'9"" tal...",205.74,basketball
1,1,Captain America has 1200 ryo. He spends 400 ry...,def solution():\n #Captain America started ...,300.0,Captain America
2,3,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream
3,5,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball
4,6,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman
...,...,...,...,...,...
2088,1901,30 unicorns are in a herd. 10 unicorns in the ...,def solution():\n #Total number of unicorns...,17,unicorns
2089,1902,A pizza has 8 slices. 4 of the slices have pep...,def solution():\n #A pizza has 8 slices\n ...,4,pizza
2090,1903,12000 people were at a rally. 10000 of them we...,def solution():\n #Total number of people a...,500,unspecified
2091,1904,A bakery made 300 muffins. 150 of the muffins ...,def solution():\n #Total muffins\n total...,60,unspecified


In [16]:
sgsm_train

,Unnamed: 0,question,solution,answer,topic
0,0,"LeBron James is 6'9"" tall. 1 inch is 2.54 cm. ...","def solution():\n #LeBron James is 6'9"" tal...",205.74,basketball
1,1,Captain America has 1200 ryo. He spends 400 ry...,def solution():\n #Captain America started ...,300.0,Captain America
2,3,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream
3,5,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball
4,6,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman
...,...,...,...,...,...
2088,1901,30 unicorns are in a herd. 10 unicorns in the ...,def solution():\n #Total number of unicorns...,17,unicorns
2089,1902,A pizza has 8 slices. 4 of the slices have pep...,def solution():\n #A pizza has 8 slices\n ...,4,pizza
2090,1903,12000 people were at a rally. 10000 of them we...,def solution():\n #Total number of people a...,500,unspecified
2091,1904,A bakery made 300 muffins. 150 of the muffins ...,def solution():\n #Total muffins\n total...,60,unspecified


In [23]:
df

,question,solution,answer,topic,subset
10739,The Egyptians built 100 pyramids. 20 of them a...,def solution():\n #Total number of pyramids...,15,Ancient Civilizations,sgsm_unannotated
5852,A 2-liter bottle of soda has 80 calories. A 1....,def solution():\n #A 2-liter bottle of soda...,400,Health and Nutrition,sgsm_unannotated
1910,There are 20000 apples in an orchard. 2000 of ...,def solution():\n #There are 20000 apples i...,15000,unspecified,sgsm_unannotated
7809,There are 1000 pages in a Mystery Stories book...,def solution():\n #Total pages in the book\...,500,Mystery Stories,sgsm_unannotated
14961,1000 Minions are in a room. 200 of them are we...,def solution():\n #1000 Minions are in a ro...,1000,Minions,sgsm_unannotated
...,...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts,sgsm_unannotated
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures,sgsm_unannotated
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens,sgsm_unannotated
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)",sgsm_unannotated


In [25]:
df['topic'].value_counts()

topic
unspecified                                          4383
National Parks                                        159
LeBron James                                          157
Dance and Ballet                                      154
cheer leading                                         154
                                                     ... 
Transformers                                           18
Up                                                     18
Pokemon                                                13
Hobbies (e.g., stamp collecting, coin collecting)      10
Pets (cats, dogs, hamsters)                             6
Name: count, Length: 198, dtype: int64

In [34]:
import random
for i in range(0, 10):
    index = random.randint(0, len(df))
    question = df.iloc[index]['question']
    solution = df.iloc[index]['solution']
    print(f"Question: {question}\n\nSolution:\n{solution}")

Question: A soccer team has 11 players. 3 of them are goalkeepers, 4 are defenders, and 4 are forwards. How many midfielders does the team have?

Solution:
def solution():
    #Total number of players
    total_players = 11
    #Number of goalkeepers, defenders, and forwards
    goalkeepers = 3
    defenders = 4
    forwards = 4
    #The answer is
    midfielders = total_players - goalkeepers - defenders - forwards
    return midfielders
Question: There are 1200 volcanoes on Earth. 

600 of them are active, 

300 of them are dormant, 

and 300 of them are extinct. 

How many volcanoes are there in each category?

Solution:
def solution():

    #Total number of volcanoes

    total_volcanoes = 1200

    #Number of active volcanoes

    active_volcanoes = 600

    #Number of dormant volcanoes

    dormant_volcanoes = 300

    #Number of extinct volcanoes

    extinct_volcanoes = 300

    #The answer is

    result = [active_volcanoes, dormant_volcanoes, extinct_volcanoes]

    return res

In [21]:
18397+2093

20490

In [36]:
import json
import random
# Read the content of the first JSON file
with open('data/mathwell_final.json', 'r') as file:
    data1 = json.load(file)

# Read the content of the second JSON file
with open('data/initial_train.json', 'r') as file:
    data2 = json.load(file)

# Merge the contents into a single list (assuming both files contain lists)
merged_data = data1 + data2

# Shuffle the merged data
random.shuffle(merged_data)

# Write the shuffled data to a new JSON file
with open('data/full_train.json', 'w') as file:
    json.dump(merged_data, file)  # Use indent for pretty formatting (optional)


## Format Annotation Data for KTO

In [37]:
annotations = pd.read_csv('data/annotations.csv')
annotations

,model,question,solution,answer,accuracy,solvability,appropriateness,mac
0,llama,Steph Curry has 3000 points. 1000 of them are ...,def solution():\n #Steph Curry has 3000 poi...,0,1.0,1,1.0,1
1,mammoth,The soccer team has 15 players. Each player ha...,def solution():\n #Number of players\n p...,30,1.0,1,1.0,1
2,llama,A commercial airplane takes 12 hours to fly fr...,def solution():\n #A commercial airplane ta...,0.6,0.0,1,1.0,0
3,llema,"In the original comic book series, the four Tu...",def solution():\n #Raphael is 6 inches tall...,30,NaN,0,NaN,0
4,llema,Spiderman has a web-shooter that shoots spider...,def solution():\n #Spiderman needs to load ...,180,1.0,1,1.0,1
...,...,...,...,...,...,...,...,...
4729,finetuned_llama,A field hockey game has 11 players per team. T...,def solution():\n #A field hockey game has ...,33,0.0,1,0.0,0
4730,finetuned_llama,100 turtles are at the pond. 10 turtles are ea...,def solution():\n #Total turtles\n total...,0,0.0,1,0.0,0
4731,finetuned_llama,Barbie has 1200 Instagram followers. 200 more ...,def solution():\n #Barbie started with 1200...,1150,0.0,0,0.0,0
4732,finetuned_llama,1234567890 is a 10-digit number. \nHow many 3-...,def solution():\n #10-digit number has 10 d...,3.3333333333333335,0.0,1,0.0,0


In [38]:
prompt = "Write a grade school math word problem and Python function with a commented out step-by-step solution to solve the word problem."
formatted_prompt = [f"Below is an instruction that describes a task. " 
                f"Write a response that appropriately completes the request.\n\n" + 
                f"### Instruction:\n{prompt}\n\n### Response: "]
formatted_prompt = "".join(formatted_prompt)
print(formatted_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a grade school math word problem and Python function with a commented out step-by-step solution to solve the word problem.

### Response: 


In [39]:
annotations = annotations.sample(frac = 1, random_state=42)
annotations['prompt'] = formatted_prompt
annotations['completion'] = "Question: " + annotations['question'] + "\nSolution:\n" + annotations['solution']
def label(df):
    if df['mac'] == 1:
        return True
    else:
        return False
annotations['label'] = annotations.apply(label, axis =1)
annotations

,model,question,solution,answer,accuracy,solvability,appropriateness,mac,prompt,completion,label
2307,finetuned_llama,The Varsity Lacrosse team has 24 players. The ...,def solution():\n #The Varsity Lacrosse tea...,30,NaN,0,0.0,0,Below is an instruction that describes a task....,Question: The Varsity Lacrosse team has 24 pla...,False
354,llema,If the Super Mario can jump over a total of 5 ...,def solution():\n #The super Mario can jump...,2.5,NaN,0,NaN,0,Below is an instruction that describes a task....,Question: If the Super Mario can jump over a t...,False
1397,gpt4,Naruto has been asked to deliver scrolls to hi...,def solution():\n # Naruto starts with 25 s...,7,1.0,1,1.0,1,Below is an instruction that describes a task....,Question: Naruto has been asked to deliver scr...,True
3366,finetuned_llama,Superman can bench press 3000 pounds. He can ...,def solution():\n #Superman can bench press...,10464.96,1.0,1,1.0,1,Below is an instruction that describes a task....,Question: Superman can bench press 3000 pounds...,True
2036,finetuned_llama,Wonder Woman is in a 10-day battle. \n\nShe fi...,def solution():\n #Wonder Woman fights 30 e...,840,NaN,0,0.0,0,Below is an instruction that describes a task....,Question: Wonder Woman is in a 10-day battle. ...,False
...,...,...,...,...,...,...,...,...,...,...,...
4426,finetuned_llama,12 unicorns ate 100 apples. 24 unicorns ate 20...,def solution():\n #12 unicorns ate 100 appl...,6000,0.0,1,1.0,0,Below is an instruction that describes a task....,Question: 12 unicorns ate 100 apples. 24 unico...,False
466,mammoth,"LeBron James has 4 MVP awards, 3 NBA Finals MV...",def solution():\n #LeBron James has 4 MVP a...,9,1.0,1,1.0,1,Below is an instruction that describes a task....,"Question: LeBron James has 4 MVP awards, 3 NBA...",True
3092,finetuned_llama,124 cars are in a parking lot. 102 of the cars...,def solution():\n #Total cars\n total_ca...,7,1.0,1,1.0,1,Below is an instruction that describes a task....,Question: 124 cars are in a parking lot. 102 o...,True
3772,finetuned_llama,Sally baked 240 cookies. She put them in 20 ba...,def solution():\n #Sally baked 240 cookies\...,12.0,1.0,1,1.0,1,Below is an instruction that describes a task....,Question: Sally baked 240 cookies. She put the...,True


In [40]:
annotations = annotations[['prompt', 'completion', 'label']]
annotations = annotations.dropna()
annotations.to_csv('data/kto.csv', index=False)
annotations

,prompt,completion,label
2307,Below is an instruction that describes a task....,Question: The Varsity Lacrosse team has 24 pla...,False
354,Below is an instruction that describes a task....,Question: If the Super Mario can jump over a t...,False
1397,Below is an instruction that describes a task....,Question: Naruto has been asked to deliver scr...,True
3366,Below is an instruction that describes a task....,Question: Superman can bench press 3000 pounds...,True
2036,Below is an instruction that describes a task....,Question: Wonder Woman is in a 10-day battle. ...,False
...,...,...,...
4426,Below is an instruction that describes a task....,Question: 12 unicorns ate 100 apples. 24 unico...,False
466,Below is an instruction that describes a task....,"Question: LeBron James has 4 MVP awards, 3 NBA...",True
3092,Below is an instruction that describes a task....,Question: 124 cars are in a parking lot. 102 o...,True
3772,Below is an instruction that describes a task....,Question: Sally baked 240 cookies. She put the...,True


In [60]:
round(annotations['label'].mean()+1, 2)

1.59

In [61]:
round((1-annotations['label'].mean())+1, 2)

1.41

In [56]:
from datasets import load_dataset, Dataset
dataset = load_dataset('csv', data_files="data/kto.csv")
dataset

/Users/bryan/.pyenv/versions/3.11.5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 0 examples [00:00, ? examples/s]/Users/bryan/.pyenv/versions/3.11.5/lib/python3.11/site-packages/datasets/download/streaming_download_manager.py:784: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 4734 examples [00:00, 133214.15 examples/s]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'label'],
        num_rows: 4734
    })
})

In [52]:
df = pd.read_csv('data/mathwell.csv')
print(df.iloc[1]['output'])

Question: A class has 120 students. 40 of them are in the 11th grade. 60 are in the 12th grade. 20 are in the 9th grade. How many students are in the 10th grade?
Solution:
def solution():
    #The total number of students is 120
    total_students = 120
    #The number of 11th grade students is 40
    eleventh_grade_students = 40
    #The number of 12th grade students is 60
    twelfth_grade_students = 60
    #The number of 9th grade students is 20
    ninth_grade_students = 20
    #The number of 10th grade students is
    tenth_grade_students = total_students - eleventh_grade_students - twelfth_grade_students - ninth_grade_students
    return tenth_grade_students


In [53]:
print(annotations.iloc[1]['completion'])

Question: Iron Man has 3000 nanobots. He uses 1000 nanobots to heal his wounds. He then uses 2000 nanobots to rebuild his armor. How many nanobots does he have left?
Solution:
def solution():
    #Iron Man started with 3000 nanobots
    nanobots_initial = 3000
    #He used 1000 to heal his wounds
    nanobots_heal = 1000
    #He used 2000 to rebuild his armor
    nanobots_armor = 2000
    #The answer is
    result = nanobots_initial - nanobots_heal - nanobots_armor
    return result


In [43]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from trl import KTOConfig, KTOTrainer, ModelConfig, get_peft_config, setup_chat_format
import pandas as pd
from collections import defaultdict
from typing import Optional, Dict, Sequence
df = pd.read_csv('data/kto.csv')
neg_label = round(df['label'].mean()+1, 2)
pos_label = round((1-df['label'].mean())+1, 2)
dataset = load_dataset('csv', data_files="data/kto.csv")
dataset = dataset['train'].train_test_split(test_size = .15, seed = 42)

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
model_ref = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
# Set special tokens
DEFAULT_PAD_TOKEN = "[PAD]"
def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel, model2: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=64)
    model2.resize_token_embeddings(len(tokenizer), pad_to_multiple_of=64)
    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg

        input_embeddings = model2.get_input_embeddings().weight.data
        output_embeddings = model2.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg
        
if tokenizer._pad_token is None:
    smart_tokenizer_and_embedding_resize(
        special_tokens_dict=dict(pad_token=DEFAULT_PAD_TOKEN),
        tokenizer=tokenizer,
        model=model, model2 = model_ref, 
    )

print('Adding special tokens.')
tokenizer.add_special_tokens({
        "eos_token": tokenizer.convert_ids_to_tokens(model.config.eos_token_id),
        "bos_token": tokenizer.convert_ids_to_tokens(model.config.bos_token_id),
#                 "unk_token": tokenizer.convert_ids_to_tokens(
#                     model.config.pad_token_id if model.config.pad_token_id != -1 else tokenizer.pad_token_id
#                 ),
})
training_args = KTOConfig(
    beta=0.1,
    desirable_weight=pos_label,
    undesirable_weight=neg_label,
    warmup_ratio = 0.1,
    num_train_epochs = 3,
    learning_rate = 1e-4, 
    per_device_train_batch_size = 8,
    output_dir="./mathwell8b-kto", 
    save_steps = 250, 
    do_eval = True,
    eval_steps = 500,
    report_to = 'wandb',
)

kto_trainer = KTOTrainer(
    model,
    model_ref,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    tokenizer=tokenizer,
)

kto_trainer.train()

/Users/bryan/.pyenv/versions/3.11.5/lib/python3.11/site-packages/trl/trainer/kto_trainer.py:417: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the KTOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/Users/bryan/.pyenv/versions/3.11.5/lib/python3.11/site-packages/trl/trainer/kto_trainer.py:427: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the KTOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/Users/bryan/.pyenv/versions/3.11.5/lib/python3.11/site-packages/trl/trainer/kto_trainer.py:457: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your KTOConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Adding special tokens.


Extracting KL train dataset: 100%|█| 4022/4022 [00:00<00:00, 9668.73 examples/s]
Processing tokenized train dataset: 100%|█| 4022/4022 [00:00<00:00, 5581.83 exam
Processing tokenized train KL dataset: 100%|█| 4022/4022 [00:00<00:00, 5936.50 e
Extracting eval KL dataset: 100%|████| 710/710 [00:00<00:00, 9051.07 examples/s]
Processing tokenized eval dataset: 100%|█| 710/710 [00:00<00:00, 5331.41 example
Processing tokenized eval KL dataset: 100%|█| 710/710 [00:00<00:00, 6114.21 exam
Filtering desirable examples: 100%|█| 4022/4022 [00:01<00:00, 2876.53 examples/s
Filtering undesirable examples: 100%|█| 4022/4022 [00:01<00:00, 2934.49 examples
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Abort: 

In [27]:
prompt_and_completion = [
    prompt + completion
    for prompt, completion in zip(dataset['train']["prompt"], dataset['train']["completion"])
    if prompt is not None and completion is None
]

TypeError: can only concatenate str (not "NoneType") to str

In [36]:
df = pd.read_csv('data/kto.csv')
df

,prompt,completion,label
0,Below is an instruction that describes a task....,Question: Patrick Mahomes has 4000 passing yar...,False
1,Below is an instruction that describes a task....,Question: There are 1000 students at a school....,True
2,Below is an instruction that describes a task....,Question: A small pizzeria sells 8 slices of p...,False
3,Below is an instruction that describes a task....,Question: A candy store sells 500 candy canes....,False
4,Below is an instruction that describes a task....,Question: Wonder Woman has 100 lassos. She las...,False
...,...,...,...
4729,Below is an instruction that describes a task....,Question: There are 20 planes in the sky. 5 ar...,True
4730,Below is an instruction that describes a task....,Question: A pizza is cut into 8 slices. 2 of t...,True
4731,Below is an instruction that describes a task....,Question: 5000 people live in a small town. 20...,False
4732,Below is an instruction that describes a task....,Question: Spiderman can shoot 3 webs per secon...,True


In [34]:
df = df.dropna()
df

,prompt,completion,label
0,Below is an instruction that describes a task....,Question: Patrick Mahomes has 4000 passing yar...,False
1,Below is an instruction that describes a task....,Question: There are 1000 students at a school....,True
2,Below is an instruction that describes a task....,Question: A small pizzeria sells 8 slices of p...,False
3,Below is an instruction that describes a task....,Question: A candy store sells 500 candy canes....,False
4,Below is an instruction that describes a task....,Question: Wonder Woman has 100 lassos. She las...,False
...,...,...,...
4729,Below is an instruction that describes a task....,Question: There are 20 planes in the sky. 5 ar...,True
4730,Below is an instruction that describes a task....,Question: A pizza is cut into 8 slices. 2 of t...,True
4731,Below is an instruction that describes a task....,Question: 5000 people live in a small town. 20...,False
4732,Below is an instruction that describes a task....,Question: Spiderman can shoot 3 webs per secon...,True


In [73]:
df = pd.read_csv('data/kto.csv')
neg_label = round(df['label'].mean()+1, 2)
pos_label = round((1-df['label'].mean())+1, 2)
dataset = load_dataset('csv', data_files="data/kto.csv")
dataset = dataset['train'].train_test_split(test_size = .05, seed = 42)

In [74]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'label'],
        num_rows: 4497
    })
    test: Dataset({
        features: ['prompt', 'completion', 'label'],
        num_rows: 237
    })
})

In [1]:
final_prompt = f"Write a grade school math word problem about pizza and Python function with a commented out step-by-step solution to solve the word problem."
final_prompt = f"Below is an instruction that describes a task. " + f"Write a response that appropriately completes the request.\n\n" + f"### Instruction:\n{final_prompt}\n\n### Response: "

In [2]:
print(final_prompt)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a grade school math word problem about pizza and Python function with a commented out step-by-step solution to solve the word problem.

### Response: 
